# WeatherPy
-----------------------------------------------------------------------------------------------------------------------------------------------------------

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [17]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 604


# Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

## Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [ ]:
import json

# Load the JSON file containing the list of city names
with open("city.list.json", "r", encoding="utf-8") as file:
    city_data = json.load(file)

# Extract city names from the loaded JSON data
cities = [city["name"] for city in city_data]

# Set the API base URL
url = "https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={weather_api_key}"

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = f"{url}q={city}&appid={weather_api_key}"
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        response = requests.get(city_url)
        city_weather = response.json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | Ḩeşār-e Sefīd
City not found. Skipping...
Processing Record 2 of Set 1 | ‘Ayn Ḩalāqīm
City not found. Skipping...
Processing Record 3 of Set 1 | Taglag
City not found. Skipping...
Processing Record 4 of Set 1 | Qabāghlū
City not found. Skipping...
Processing Record 5 of Set 1 | ‘Arīqah
City not found. Skipping...
Processing Record 6 of Set 1 | Kalāteh-ye Dowlat
City not found. Skipping...
Processing Record 7 of Set 1 | Behjatābād
City not found. Skipping...
Processing Record 8 of Set 1 | Ţālesh Maḩalleh
City not found. Skipping...
Processing Record 9 of Set 1 | Shahrīār Kandeh
City not found. Skipping...
Processing Record 10 of Set 1 | Bālā Aḩmad Kolā
City not found. Skipping...
Processing Record 11 of Set 1 | Aş Şūrah aş Şaghīrah
City not found. Skipping...
Processing Record 12 of Set 1 | Āqdūz
City not found. Skipping...
Processing Record 13 of Set 1 | Āzādshahr
City not found. Skipping...
Proc